# Kaggle Competition Dogs vs Cats Classification:

_Competition link: https://www.kaggle.com/c/dogs-vs-cats/overview_

## Libraries:

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, LSTM, Conv2D, Dropout
import os, shutil

## Data Importation:

In [2]:
dir = '/Users/spavot/Documents/Perso/Kaggle-Cat-vs-Dog-Classification/Data'
original_dir = '/Users/spavot/Documents/Perso/Kaggle-Cat-vs-Dog-Classification/Data/Original_Data'
train_dir = os.path.join(dir, 'Training')
os.mkdir(train_dir)
validation_dir = os.path.join(dir, 'Validation')
os.mkdir(validation_dir)
test_dir = os.path.join(dir, 'Test')
os.mkdir(test_dir)

In [3]:
#Training set directory
train_cats_dir = os.path.join(train_dir, 'Cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'Dogs')
os.mkdir(train_dogs_dir)
#Validation set directory
validation_cats_dir = os.path.join(validation_dir, 'Cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'Dogs')
os.mkdir(validation_dogs_dir)
#Test set directory
test_cats_dir = os.path.join(test_dir, 'Cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'Dogs')
os.mkdir(test_dogs_dir)

In [4]:
#Copy the image to the right directory:
fnames = ['cat.{}.jpg'.format(i) for i in range(8000)]
for fname in fnames:
    src = os.path.join(original_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src,dst)
#Dogs images:
fnames = ['dog.{}.jpg'.format(i) for i in range(8000)]
for fname in fnames:
    src = os.path.join(original_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src,dst)

In [5]:
#Copy the image to the validation directory:
#Cats images
fnames = ['cat.{}.jpg'.format(i) for i in range(8000,10000)]
for fname in fnames:
    src = os.path.join(original_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src,dst)
#Dogs images:
fnames = ['dog.{}.jpg'.format(i) for i in range(8000,10000)]
for fname in fnames:
    src = os.path.join(original_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src,dst)

In [6]:
#Copy the image to the test directory:
#Cats images
fnames = ['cat.{}.jpg'.format(i) for i in range(10000,12500)]
for fname in fnames:
    src = os.path.join(original_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src,dst)
#Dogs images:
fnames = ['dog.{}.jpg'.format(i) for i in range(10000,12500)]
for fname in fnames:
    src = os.path.join(original_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src,dst)

In [7]:
#Check if we have the correct number of pictures per set:
print('Total training set cats:', len(os.listdir(train_cats_dir)))
print('Total training set dogs:', len(os.listdir(train_dogs_dir)))
print('Total validation set cats:', len(os.listdir(validation_cats_dir)))
print('Total validation set dogs:', len(os.listdir(validation_dogs_dir)))
print('Total test set cats:', len(os.listdir(test_cats_dir)))
print('Total test set dogs:', len(os.listdir(test_dogs_dir)))

Total training set cats: 8000
Total training set dogs: 8000
Total validation set cats: 2000
Total validation set dogs: 2000
Total test set cats: 2500
Total test set dogs: 2500


As expected, we have 8000 training samples of each class, 2000 for validation set and finally 2500 for the testing set to confirm the results and avoid overfitting on the validation set.